In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

user = 'accounting'
pw = 'unicorn'
host = 'nextcloud-do'
port = '5432'
db = 'accounting'

uri = f"postgresql+psycopg2://{quote_plus(user)}:{quote_plus(pw)}@{host}:{port}/{db}"
engine = create_engine(uri)

In [ ]:
import numpy as np

query = "SELECT * FROM analytics.transactions;"

# Execute the query and load the data into a DataFrame
df = pd.read_sql_query(query, engine)
df = df[~df['category'].isin(['Expenses:TODO', 'Income:TODO', None])]

df['account_desc'] = df['account'] + " " + df['description']
df.sample(5)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = df['account_desc'].astype(str)
y = df['category']

# Preprocess your data
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(X)
y = df['category']  # Assuming 'category' is the target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

# Train a classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Make predictions and evaluate
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
unique_values = df['category'].unique()
print(unique_values)

In [ ]:
pd.concat([pd.Series(y_test), pd.Series(predictions)], axis=1)